<a href="https://colab.research.google.com/github/Ibar-Dev/Gestor_Componentes/blob/main/Necesito_que_me_ayudes_a_organizar_el_c%C3%B3digo_que_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

¡Claro que sí! Organizar el código en clases bien definidas y un script principal es una excelente práctica para mejorar la mantenibilidad y la claridad de tu proyecto. 👍

Aquí te presento la estructura que propongo y el código para cada archivo:

**Estructura de Archivos Sugerida:**

1.  `config.py`: Para constantes y configuraciones (como `CONFIG_FILE` y `MAPEO_MAGNITUDES_PREDEFINIDO`).
2.  `enums.py`: Para la enumeración `OrigenResultados`.
3.  `utilidades.py`: Para las clases `ExtractorMagnitud` y `ManejadorExcel`.
4.  `motor_busqueda.py`: Para la clase `MotorBusqueda`.
5.  `interfaz_grafica.py`: Para la clase `InterfazGrafica`.
6.  `main.py`: El script principal que une todo.

---
## 1. `config.py`

In [ ]:
# config.py
from typing import Dict, List

CONFIG_FILE_NAME = "config_buscador_v0_7_1_mapeo.json"
LOG_FILE_NAME = "buscador_app_v0_7_1_mapeo.log"

# Mapeo de magnitudes predefinido, movido aquí para centralizar configuración.
MAPEO_MAGNITUDES_PREDEFINIDO: Dict[str, List[str]] = {
    "AMPERIOS": ["A", "AMP", "AMPS"],
    "VOLTIOS": ["V", "VA", "VAC", "VC", "VCC", "VCD", "VDC"],
    "VATIOS": ["W", "WATTS"],
    "GIGABIT": ["G", "GB", "GBE", "GE", "GBIT"],
    "PUERTO": ["P", "PORT", "PORTS", "PTOS"],
    "HERTZ": ["HZ", "KHZ", "MHZ", "GHZ"],
    "AH": [], "ANTENNA": [], "BASE": [], "BIT": [], "ETH": [], "FE": [],
    "GBASE": [], "GBASEWAN": [], "GBIC": [], "GBPS": [], "GH": [], "KM": [],
    "KVA": [], "KW": [], "LINEAS": [], "LINES": [], "NM": [], "E": [],
    "POTS": [], "STM": []
}

# Otros valores de configuración que puedan ser necesarios
# Ejemplo:
# DEFAULT_WINDOW_GEOMETRY = "1250x800"
# DEFAULT_LOG_LEVEL = "INFO" # o logging.INFO si se importa logging aquí

---
## 2. `enums.py`

In [ ]:
# enums.py
from enum import Enum, auto

class OrigenResultados(Enum):
    NINGUNO = 0
    VIA_DICCIONARIO_CON_RESULTADOS_DESC = auto()
    VIA_DICCIONARIO_SIN_TERMINOS_VALIDOS = auto()
    VIA_DICCIONARIO_SIN_RESULTADOS_DESC = auto()
    DIRECTO_DESCRIPCION_CON_RESULTADOS = auto()
    DIRECTO_DESCRIPCION_VACIA = auto()
    ERROR_CARGA_DICCIONARIO = auto()
    ERROR_CARGA_DESCRIPCION = auto()
    ERROR_CONFIGURACION_COLUMNAS_DICC = auto()
    ERROR_CONFIGURACION_COLUMNAS_DESC = auto()
    ERROR_BUSQUEDA_INTERNA_MOTOR = auto()
    TERMINO_INVALIDO = auto()

    @property
    def es_via_diccionario(self) -> bool:
        return self in {
            OrigenResultados.VIA_DICCIONARIO_CON_RESULTADOS_DESC,
            OrigenResultados.VIA_DICCIONARIO_SIN_TERMINOS_VALIDOS,
            OrigenResultados.VIA_DICCIONARIO_SIN_RESULTADOS_DESC,
        }

    @property
    def es_directo_descripcion(self) -> bool:
        return self in {
            OrigenResultados.DIRECTO_DESCRIPCION_CON_RESULTADOS,
            OrigenResultados.DIRECTO_DESCRIPCION_VACIA,
        }

    @property
    def es_error_carga(self) -> bool:
        return self in {
            OrigenResultados.ERROR_CARGA_DICCIONARIO,
            OrigenResultados.ERROR_CARGA_DESCRIPCION,
        }

    @property
    def es_error_configuracion(self) -> bool:
        return self in {
            OrigenResultados.ERROR_CONFIGURACION_COLUMNAS_DICC,
            OrigenResultados.ERROR_CONFIGURACION_COLUMNAS_DESC,
        }

    @property
    def es_error_operacional(self) -> bool:
        return self == OrigenResultados.ERROR_BUSQUEDA_INTERNA_MOTOR

    @property
    def es_termino_invalido(self) -> bool:
        return self == OrigenResultados.TERMINO_INVALIDO

---
## 3. `utilidades.py`

In [ ]:
# utilidades.py
import unicodedata
import pandas as pd
from pathlib import Path
import logging
from typing import Dict, List, Optional, Tuple, Union

# Importar configuración
from config import MAPEO_MAGNITUDES_PREDEFINIDO

logger = logging.getLogger(__name__)

class ExtractorMagnitud:
    def __init__(self, mapeo_magnitudes: Optional[Dict[str, List[str]]] = None):
        self.sinonimo_a_canonico_normalizado: Dict[str, str] = {}
        # Usar el mapeo de config.py si no se proporciona uno específico
        mapeo_a_usar = mapeo_magnitudes if mapeo_magnitudes is not None else MAPEO_MAGNITUDES_PREDEFINIDO

        for forma_canonica, lista_sinonimos in mapeo_a_usar.items():
            canonico_norm = self._normalizar_texto(forma_canonica)
            if not canonico_norm:
                logger.warning(f"Forma canónica '{forma_canonica}' resultó vacía tras normalizar. Se ignora.")
                continue

            self.sinonimo_a_canonico_normalizado[canonico_norm] = canonico_norm

            for sinonimo in lista_sinonimos:
                sinonimo_norm = self._normalizar_texto(sinonimo)
                if sinonimo_norm:
                    if sinonimo_norm in self.sinonimo_a_canonico_normalizado and \
                       self.sinonimo_a_canonico_normalizado[sinonimo_norm] != canonico_norm:
                        logger.warning(
                            f"Conflicto de mapeo: El sinónimo normalizado '{sinonimo_norm}' (de '{sinonimo}' para '{forma_canonica}') "
                            f"ya está mapeado a '{self.sinonimo_a_canonico_normalizado[sinonimo_norm]}'. "
                            f"Se sobrescribirá con el mapeo a '{canonico_norm}'. "
                            "Revise su MAPEO_MAGNITUDES_PREDEFINIDO para evitar ambigüedades."
                        )
                    self.sinonimo_a_canonico_normalizado[sinonimo_norm] = canonico_norm
        logger.debug(f"ExtractorMagnitud inicializado con mapeo: {self.sinonimo_a_canonico_normalizado}")

    @staticmethod
    def _normalizar_texto(texto: str) -> str:
        if not isinstance(texto, str) or not texto:
            return ""
        try:
            texto_upper = texto.upper()
            forma_normalizada = unicodedata.normalize("NFKD", texto_upper)
            return "".join(c for c in forma_normalizada if not unicodedata.combining(c))
        except TypeError:
            return ""

    def obtener_magnitud_normalizada(self, texto_unidad: str) -> Optional[str]:
        if not texto_unidad:
            return None
        normalizada = self._normalizar_texto(texto_unidad)
        if not normalizada:
            return None
        return self.sinonimo_a_canonico_normalizado.get(normalizada)


class ManejadorExcel:
    @staticmethod
    def cargar_excel(
        ruta_archivo: Union[str, Path],
    ) -> Tuple[Optional[pd.DataFrame], Optional[str]]:
        ruta = Path(ruta_archivo)
        logger.info(f"Intentando cargar archivo Excel: {ruta}")
        if not ruta.exists():
            error_msg = f"¡Archivo no encontrado! Ruta: {ruta}"
            logger.error(error_msg)
            return None, error_msg
        try:
            engine = "openpyxl" if ruta.suffix.lower() == ".xlsx" else None
            df = pd.read_excel(ruta, engine=engine)
            logger.info(f"Archivo '{ruta.name}' cargado ({len(df)} filas).")
            return df, None
        except Exception as e:
            error_msg = (
                f"No se pudo cargar el archivo:\n{ruta}\n\nError: {e}\n\n"
                "Posibles causas:\n"
                "- El archivo está siendo usado por otro programa.\n"
                "- No tiene instalado 'openpyxl' para .xlsx (o 'xlrd' para .xls).\n"
                "- El archivo está corrupto o en formato no soportado."
            )
            logger.exception(f"Error inesperado al cargar archivo Excel: {ruta}")
            return None, error_msg

---
## 4. `motor_busqueda.py`

In [ ]:
# motor_busqueda.py
import re
import pandas as pd
from pathlib import Path
import logging
from typing import Optional, List, Tuple, Dict, Any, Set, Union

from enums import OrigenResultados
from utilidades import ExtractorMagnitud, ManejadorExcel

logger = logging.getLogger(__name__)

class MotorBusqueda:
    def __init__(self, indices_diccionario_cfg: Optional[List[int]] = None):
        self.datos_diccionario: Optional[pd.DataFrame] = None
        self.datos_descripcion: Optional[pd.DataFrame] = None
        self.archivo_diccionario_actual: Optional[Path] = None
        self.archivo_descripcion_actual: Optional[Path] = None

        self.indices_columnas_busqueda_dic: List[int] = (
            indices_diccionario_cfg if isinstance(indices_diccionario_cfg, list) else []
        )
        logger.info(
            f"MotorBusqueda inicializado. Índices búsqueda diccionario: {self.indices_columnas_busqueda_dic or 'Todas las de texto'}"
        )

        self.patron_comparacion = re.compile(
            r"^([<>]=?)(\d+(?:[.,]\d+)?)\s*([a-zA-ZáéíóúÁÉÍÓÚñÑµΩ]+)?(.*)$"
        )
        self.patron_rango = re.compile(
            r"^(\d+(?:[.,]\d+)?)\s*-\s*(\d+(?:[.,]\d+)?)\s*([a-zA-ZáéíóúÁÉÍÓÚñÑµΩ]+)?$"
        )
        self.patron_negacion = re.compile(r"^#(.+)$")
        self.patron_num_unidad_df = re.compile(
            r"(\d+(?:[.,]\d+)?)\s*([a-zA-ZáéíóúÁÉÍÓÚñÑµΩ]+)?"
        )
        self.extractor_magnitud = ExtractorMagnitud() # Usa el mapeo por defecto de config.py

    def cargar_excel_diccionario(self, ruta_str: str) -> Tuple[bool, Optional[str]]:
        ruta = Path(ruta_str)
        df_cargado, error_msg_carga = ManejadorExcel.cargar_excel(ruta)

        if df_cargado is None:
            self.datos_diccionario = None
            self.archivo_diccionario_actual = None
            return False, error_msg_carga or "Error desconocido al cargar diccionario."

        valido, msg_val_cols = self._validar_columnas_df(
            df_cargado, self.indices_columnas_busqueda_dic, "diccionario"
        )
        if not valido:
            logger.warning(
                f"Validación de columnas del diccionario fallida. Carga invalidada. Causa: {msg_val_cols}"
            )
            return False, msg_val_cols or "Validación de columnas del diccionario fallida."

        self.datos_diccionario = df_cargado
        self.archivo_diccionario_actual = ruta
        return True, None

    def cargar_excel_descripcion(self, ruta_str: str) -> Tuple[bool, Optional[str]]:
        ruta = Path(ruta_str)
        df_cargado, error_msg_carga = ManejadorExcel.cargar_excel(ruta)

        if df_cargado is None:
            self.datos_descripcion = None
            self.archivo_descripcion_actual = None
            return False, error_msg_carga or "Error desconocido al cargar descripciones."

        self.datos_descripcion = df_cargado
        self.archivo_descripcion_actual = ruta
        return True, None

    def _validar_columnas_df(
        self, df: Optional[pd.DataFrame], indices_cfg: List[int], nombre_df_log: str
    ) -> Tuple[bool, Optional[str]]:
        if df is None:
            msg = f"DataFrame '{nombre_df_log}' es None, no se puede validar."
            logger.error(msg)
            return False, msg

        num_cols_df = len(df.columns)

        if not indices_cfg or indices_cfg == [-1]:
            if num_cols_df == 0:
                msg = f"El archivo del {nombre_df_log} está vacío o no contiene columnas (modo 'todas')."
                logger.error(msg)
                return False, msg
            return True, None

        if not all(isinstance(idx, int) and idx >= 0 for idx in indices_cfg):
            msg = f"Configuración de índices para {nombre_df_log} inválida: {indices_cfg}. Deben ser enteros no negativos."
            logger.error(msg)
            return False, msg

        max_indice_requerido = max(indices_cfg) if indices_cfg else -1

        if num_cols_df == 0:
            msg = f"El {nombre_df_log} no tiene columnas."
            logger.error(msg)
            return False, msg
        elif max_indice_requerido >= num_cols_df:
            msg = (
                f"El {nombre_df_log} necesita al menos {max_indice_requerido + 1} columnas "
                f"para los índices configurados ({indices_cfg}), pero solo tiene {num_cols_df}."
            )
            logger.error(msg)
            return False, msg
        return True, None

    def _obtener_nombres_columnas_busqueda_df(
        self, df: Optional[pd.DataFrame], indices_cfg: List[int], nombre_df_log: str
    ) -> Tuple[Optional[List[str]], Optional[str]]:
        if df is None:
            msg = f"Intentando obtener nombres de columnas de un DataFrame ({nombre_df_log}) que es None."
            logger.error(msg)
            return None, msg

        columnas_disponibles = df.columns
        num_cols_df = len(columnas_disponibles)

        if not indices_cfg or indices_cfg == [-1]:
            cols_texto_obj = [
                col for col in df.columns
                if pd.api.types.is_string_dtype(df[col]) or pd.api.types.is_object_dtype(df[col])
            ]
            if cols_texto_obj:
                logger.info(
                    f"Buscando en columnas de texto/object (detectadas) del {nombre_df_log}: {cols_texto_obj}"
                )
                return cols_texto_obj, None
            elif num_cols_df > 0:
                logger.warning(
                    f"No se encontraron columnas de texto/object en {nombre_df_log}. Se usarán todas las {num_cols_df} columnas."
                )
                return list(df.columns), None
            else:
                msg = f"El DataFrame del {nombre_df_log} no tiene columnas."
                logger.error(msg)
                return None, msg

        nombres_columnas_seleccionadas = []
        indices_validos_usados = []
        for indice in indices_cfg:
            if 0 <= indice < num_cols_df:
                nombres_columnas_seleccionadas.append(columnas_disponibles[indice])
                indices_validos_usados.append(indice)
            else:
                logger.warning(
                    f"Índice {indice} para {nombre_df_log} es inválido o fuera de rango (0 a {num_cols_df-1}). Ignorado."
                )

        if not nombres_columnas_seleccionadas:
            msg = f"No se encontraron columnas válidas en {nombre_df_log} con los índices configurados: {indices_cfg}"
            logger.error(msg)
            return None, msg

        logger.debug(
            f"Se buscará en columnas del {nombre_df_log}: {nombres_columnas_seleccionadas} (índices: {indices_validos_usados})"
        )
        return nombres_columnas_seleccionadas, None

    def _parsear_nivel1_or(self, texto_complejo: str) -> Tuple[str, List[str]]:
        texto_limpio = texto_complejo.strip()
        if not texto_limpio:
            return 'OR', []

        if '|' in texto_limpio:
            segmentos = [s.strip() for s in re.split(r'\s*\|\s*', texto_limpio) if s.strip()]
            return 'OR', segmentos
        elif '/' in texto_limpio:
            segmentos = [s.strip() for s in re.split(r'\s*/\s*', texto_limpio) if s.strip()]
            return 'OR', segmentos
        else:
            return 'AND', [texto_limpio]

    def _parsear_nivel2_and(self, termino_segmento_n1: str) -> Tuple[str, List[str]]:
        termino_limpio = termino_segmento_n1.strip()
        if not termino_limpio:
            return 'AND', []

        op_principal_interno = 'AND'
        separador_interno = None

        if '+' in termino_limpio:
            separador_interno = '+'

        terminos_brutos_finales = []
        if separador_interno:
            estado = 0
            termino_actual_maquina = []
            pos = 0
            while pos < len(termino_limpio):
                char = termino_limpio[pos]
                if estado == 0:
                    if char == separador_interno:
                        sub_termino = "".join(termino_actual_maquina).strip()
                        if sub_termino: terminos_brutos_finales.append(sub_termino)
                        termino_actual_maquina = []
                    elif char in "<>=":
                        estado = 1
                        termino_actual_maquina.append(char)
                    elif char.isdigit():
                        estado = 2
                        termino_actual_maquina.append(char)
                    else:
                        termino_actual_maquina.append(char)
                elif estado == 1:
                    termino_actual_maquina.append(char)
                    if char.isdigit() or char == ".":
                        estado = 2
                    elif char.isspace() and not any(c in "<>=" for c in termino_actual_maquina[-2:]):
                        if "".join(termino_actual_maquina).strip() in ['<','>','<=','>=','=']:
                           pass
                        else:
                           estado = 0
                    elif not (char in "<>=" or char.isalnum() or char in ['.',',','-']):
                        estado = 0
                elif estado == 2:
                    termino_actual_maquina.append(char)
                    if not (char.isdigit() or char in ['.', ',']):
                        if char.isalpha():
                            estado = 3
                        else:
                            estado = 0
                elif estado == 3:
                    termino_actual_maquina.append(char)
                    if not char.isalnum():
                        estado = 0
                pos += 1

            sub_termino_final = "".join(termino_actual_maquina).strip()
            if sub_termino_final: terminos_brutos_finales.append(sub_termino_final)

            if not terminos_brutos_finales and termino_limpio == separador_interno:
                return op_principal_interno, []
        else:
            terminos_brutos_finales = [termino_limpio]

        return op_principal_interno, [t for t in terminos_brutos_finales if t]

    def _analizar_terminos(self, terminos_brutos: List[str]) -> List[Dict[str, Any]]:
        palabras_analizadas = []
        for term_orig_bruto in terminos_brutos:
            term_orig = str(term_orig_bruto)
            term = term_orig.strip()
            if not term: continue

            item_analizado: Dict[str, Any] = {'original': term_orig, 'negate': False}
            match_neg = self.patron_negacion.match(term)
            if match_neg:
                item_analizado['negate'] = True
                term = match_neg.group(1).strip()
                if not term: continue

            match_comp = self.patron_comparacion.match(term)
            match_range = self.patron_rango.match(term)

            if match_comp:
                op, v_str, unidad_str, _ = match_comp.groups()
                v_num = self._parse_numero(v_str)
                if v_num is not None:
                    op_map = {'>': 'gt', '<': 'lt', '>=': 'ge', '<=': 'le', '=': 'eq'}
                    unidad_canon_comp = None
                    if unidad_str:
                        unidad_canon_comp = self.extractor_magnitud.obtener_magnitud_normalizada(unidad_str.strip())
                        if unidad_canon_comp is None:
                            logger.warning(
                                f"Unidad de búsqueda '{unidad_str.strip()}' en '{term}' no reconocida. "
                                "Comparación numérica sin filtro de unidad."
                            )
                    item_analizado.update({
                        'tipo': op_map.get(op, 'str'),
                        'valor': v_num,
                        'unidad_busqueda': unidad_canon_comp
                    })
                else:
                    item_analizado.update({'tipo': 'str', 'valor': self.extractor_magnitud._normalizar_texto(term)})
            elif match_range:
                v1_str, v2_str, unidad_rango_str = match_range.groups()
                v1, v2 = self._parse_numero(v1_str), self._parse_numero(v2_str)
                if v1 is not None and v2 is not None:
                    unidad_canon_range = None
                    if unidad_rango_str:
                        unidad_canon_range = self.extractor_magnitud.obtener_magnitud_normalizada(unidad_rango_str.strip())
                        if unidad_canon_range is None:
                            logger.warning(
                                f"Unidad de rango '{unidad_rango_str.strip()}' en '{term}' no reconocida. "
                                "Rango sin filtro de unidad."
                            )
                    item_analizado.update({
                        'tipo': 'range',
                        'valor': sorted([v1, v2]),
                        'unidad_busqueda': unidad_canon_range
                    })
                else:
                    item_analizado.update({'tipo': 'str', 'valor': self.extractor_magnitud._normalizar_texto(term)})
            else:
                item_analizado.update({'tipo': 'str', 'valor': self.extractor_magnitud._normalizar_texto(term)})
            palabras_analizadas.append(item_analizado)
        logger.debug(f"Términos analizados (motor): {palabras_analizadas}")
        return palabras_analizadas

    def _parse_numero(self, num_str: Any) -> Optional[float]:
        if not isinstance(num_str, (str, int, float)): return None
        try:
            return float(str(num_str).replace(',', '.'))
        except ValueError:
            return None

    def _generar_mascara_para_un_termino(self, df: pd.DataFrame, cols_a_buscar: List[str], termino_analizado: Dict[str, Any]) -> pd.Series:
        mascara_total_subtermino = pd.Series(False, index=df.index)
        tipo_sub = termino_analizado['tipo']
        valor_sub = termino_analizado['valor']
        unidad_sub_requerida_canon = termino_analizado.get('unidad_busqueda')
        es_negado = termino_analizado.get('negate', False)

        for col_nombre in cols_a_buscar:
            if col_nombre not in df.columns:
                logger.warning(f"Columna '{col_nombre}' no encontrada en DF. Saltando.")
                continue
            col_series = df[col_nombre]
            mascara_col_actual = pd.Series(False, index=df.index)

            if tipo_sub in ['gt', 'lt', 'ge', 'le', 'range', 'eq']:
                for idx, valor_celda in col_series.items():
                    if pd.isna(valor_celda) or str(valor_celda).strip() == "": continue

                    for match_celda in self.patron_num_unidad_df.finditer(str(valor_celda)):
                        try:
                            num_celda_val = float(match_celda.group(1).replace(',', '.'))
                            unidad_celda_canon: Optional[str] = None
                            if match_celda.group(2):
                                unidad_celda_canon = self.extractor_magnitud.obtener_magnitud_normalizada(match_celda.group(2))

                            unidad_coincide = False
                            if unidad_sub_requerida_canon is None:
                                unidad_coincide = True
                            elif unidad_celda_canon is not None and unidad_celda_canon == unidad_sub_requerida_canon:
                                unidad_coincide = True

                            if not unidad_coincide:
                                continue

                            cond_ok = False
                            if tipo_sub == 'eq' and num_celda_val == valor_sub : cond_ok = True
                            elif tipo_sub == 'gt' and num_celda_val > valor_sub: cond_ok = True
                            elif tipo_sub == 'lt' and num_celda_val < valor_sub: cond_ok = True
                            elif tipo_sub == 'ge' and num_celda_val >= valor_sub: cond_ok = True
                            elif tipo_sub == 'le' and num_celda_val <= valor_sub: cond_ok = True
                            elif tipo_sub == 'range' and valor_sub[0] <= num_celda_val <= valor_sub[1]: cond_ok = True

                            if cond_ok:
                                mascara_col_actual.at[idx] = True
                                break
                        except ValueError:
                            continue
                    if mascara_col_actual.at[idx]: continue

            elif tipo_sub == 'str':
                termino_regex_escapado = r"\b" + re.escape(str(valor_sub)) + r"\b"
                try:
                    serie_normalizada = col_series.astype(str).map(self.extractor_magnitud._normalizar_texto)
                    mascara_col_actual = serie_normalizada.str.contains(termino_regex_escapado, regex=True, na=False)
                except Exception as e_conv_str:
                    logger.warning(f"No se pudo convertir/buscar string en columna '{col_nombre}': {e_conv_str}")

            mascara_total_subtermino |= mascara_col_actual.fillna(False)

        return ~mascara_total_subtermino if es_negado else mascara_total_subtermino

    def _aplicar_mascara_combinada_para_segmento_and(
        self, df: pd.DataFrame, cols_a_buscar: List[str], terminos_analizados_segmento: List[Dict[str, Any]]
    ) -> pd.Series:
        if df is None or df.empty or not cols_a_buscar:
            return pd.Series(False, index=df.index if df is not None else None)

        if not terminos_analizados_segmento:
            return pd.Series(False, index=df.index)

        mascara_final_segmento_and = pd.Series(True, index=df.index)

        for termino_individual_analizado in terminos_analizados_segmento:
            mascara_este_termino = self._generar_mascara_para_un_termino(df, cols_a_buscar, termino_individual_analizado)
            mascara_final_segmento_and &= mascara_este_termino

        return mascara_final_segmento_and

    def _combinar_mascaras_de_segmentos_or(self, lista_mascaras_segmentos: List[pd.Series], df_index_ref: pd.Index) -> pd.Series:
        if not lista_mascaras_segmentos:
            return pd.Series(False, index=df_index_ref)

        mascara_final_or = pd.Series(False, index=lista_mascaras_segmentos[0].index)
        for mascara_segmento in lista_mascaras_segmentos:
            mascara_final_or |= mascara_segmento
        return mascara_final_or

    def _procesar_busqueda_en_df_objetivo(
        self,
        df_objetivo: pd.DataFrame,
        cols_objetivo: List[str],
        termino_busqueda_original: str
    ) -> Tuple[pd.DataFrame, Optional[str]]:

        if not termino_busqueda_original.strip():
            return df_objetivo.copy(), None

        op_nivel1, segmentos_nivel1 = self._parsear_nivel1_or(termino_busqueda_original)

        if not segmentos_nivel1:
            return pd.DataFrame(columns=df_objetivo.columns), "Término de búsqueda inválido o vacío tras parseo OR."

        lista_mascaras_para_or = []
        for seg_n1 in segmentos_nivel1:
            op_nivel2, terminos_brutos_n2 = self._parsear_nivel2_and(seg_n1)

            terminos_atomicos_analizados = self._analizar_terminos(terminos_brutos_n2)

            if not terminos_atomicos_analizados:
                logger.warning(f"Segmento OR '{seg_n1}' no produjo términos analizables. No contribuirá.")
                mascara_segmento_n1 = pd.Series(False, index=df_objetivo.index)
            else:
                mascara_segmento_n1 = self._aplicar_mascara_combinada_para_segmento_and(
                    df_objetivo, cols_objetivo, terminos_atomicos_analizados
                )
            lista_mascaras_para_or.append(mascara_segmento_n1)

        if not lista_mascaras_para_or:
             return pd.DataFrame(columns=df_objetivo.columns), "Ningún segmento de búsqueda produjo resultados."

        mascara_final_df_objetivo = self._combinar_mascaras_de_segmentos_or(lista_mascaras_para_or, df_objetivo.index)
        return df_objetivo[mascara_final_df_objetivo].copy(), None

    def buscar(
        self,
        termino_busqueda_original: str,
        buscar_via_diccionario_flag: bool,
    ) -> Tuple[Optional[pd.DataFrame], OrigenResultados, Optional[pd.DataFrame], Optional[str]]:
        logger.info(
            f"Motor.buscar: termino='{termino_busqueda_original}', via_dicc={buscar_via_diccionario_flag}"
        )

        fcds_obtenidos: Optional[pd.DataFrame] = None
        df_vacio_desc = pd.DataFrame(columns=self.datos_descripcion.columns if self.datos_descripcion is not None else [])

        if not termino_busqueda_original.strip():
            if self.datos_descripcion is not None:
                return self.datos_descripcion.copy(), OrigenResultados.DIRECTO_DESCRIPCION_VACIA, None, None
            return df_vacio_desc, OrigenResultados.ERROR_CARGA_DESCRIPCION, None, "Descripciones no cargadas."

        if buscar_via_diccionario_flag:
            if self.datos_diccionario is None:
                return None, OrigenResultados.ERROR_CARGA_DICCIONARIO, None, "Diccionario no cargado."

            cols_dic, err_cols_dic = self._obtener_nombres_columnas_busqueda_df(
                self.datos_diccionario, self.indices_columnas_busqueda_dic, "diccionario (búsqueda)"
            )
            if not cols_dic:
                return None, OrigenResultados.ERROR_CONFIGURACION_COLUMNAS_DICC, None, err_cols_dic

            try:
                fcds_obtenidos, err_proc_dic = self._procesar_busqueda_en_df_objetivo(
                    self.datos_diccionario, cols_dic, termino_busqueda_original
                )
                if err_proc_dic:
                     return None, OrigenResultados.TERMINO_INVALIDO, None, err_proc_dic
                logger.info(f"FCDs: {len(fcds_obtenidos) if fcds_obtenidos is not None else 0}")

            except Exception as e:
                logger.exception("Error buscando en diccionario.")
                return None, OrigenResultados.ERROR_BUSQUEDA_INTERNA_MOTOR, None, f"Error interno (dic): {e}"

            if self.datos_descripcion is None:
                return None, OrigenResultados.ERROR_CARGA_DESCRIPCION, fcds_obtenidos, "Descripciones no cargadas."

            if fcds_obtenidos is None or fcds_obtenidos.empty:
                return df_vacio_desc, OrigenResultados.VIA_DICCIONARIO_SIN_RESULTADOS_DESC, fcds_obtenidos, None

            terms_fcd = self._extraer_terminos_diccionario(fcds_obtenidos, cols_dic)
            if not terms_fcd:
                return df_vacio_desc, OrigenResultados.VIA_DICCIONARIO_SIN_TERMINOS_VALIDOS, fcds_obtenidos, None

            try:
                term_or_desc = " | ".join(terms_fcd)
                cols_desc_fcd, err_cols_desc_fcd = self._obtener_nombres_columnas_busqueda_df(
                     self.datos_descripcion, [], "descripciones (vía FCDs)"
                )
                if not cols_desc_fcd:
                    return None, OrigenResultados.ERROR_CONFIGURACION_COLUMNAS_DESC, fcds_obtenidos, err_cols_desc_fcd

                res_desc_via_dic, err_proc_desc_fcd = self._procesar_busqueda_en_df_objetivo(
                    self.datos_descripcion, cols_desc_fcd, term_or_desc
                )
                if err_proc_desc_fcd:
                    return None, OrigenResultados.ERROR_BUSQUEDA_INTERNA_MOTOR, fcds_obtenidos, f"Error (desc vía FCD): {err_proc_desc_fcd}"

            except Exception as e:
                logger.exception("Error buscando términos de FCDs en descripciones.")
                return None, OrigenResultados.ERROR_BUSQUEDA_INTERNA_MOTOR, fcds_obtenidos, f"Error interno (desc vía FCD): {e}"

            if res_desc_via_dic is None or res_desc_via_dic.empty:
                return df_vacio_desc, OrigenResultados.VIA_DICCIONARIO_SIN_RESULTADOS_DESC, fcds_obtenidos, None
            return res_desc_via_dic, OrigenResultados.VIA_DICCIONARIO_CON_RESULTADOS_DESC, fcds_obtenidos, None

        else: # Búsqueda directa
            if self.datos_descripcion is None:
                return None, OrigenResultados.ERROR_CARGA_DESCRIPCION, None, "Descripciones no cargadas."

            cols_desc_directo, err_cols_desc_directo = self._obtener_nombres_columnas_busqueda_df(
                self.datos_descripcion, [], "descripciones (directa)"
            )
            if not cols_desc_directo:
                return None, OrigenResultados.ERROR_CONFIGURACION_COLUMNAS_DESC, None, err_cols_desc_directo

            try:
                res_directos, err_proc_desc_directo = self._procesar_busqueda_en_df_objetivo(
                    self.datos_descripcion, cols_desc_directo, termino_busqueda_original
                )
                if err_proc_desc_directo:
                    return None, OrigenResultados.TERMINO_INVALIDO, None, err_proc_desc_directo
                logger.info(f"Resultados directos: {len(res_directos) if res_directos is not None else 0}")
            except Exception as e:
                logger.exception("Error buscando directamente en descripciones.")
                return None, OrigenResultados.ERROR_BUSQUEDA_INTERNA_MOTOR, None, f"Error interno (desc directa): {e}"

            if res_directos is None or res_directos.empty:
                return df_vacio_desc, OrigenResultados.DIRECTO_DESCRIPCION_VACIA, None, None
            return res_directos, OrigenResultados.DIRECTO_DESCRIPCION_CON_RESULTADOS, None, None

    def _extraer_terminos_diccionario(self, df_coincidencias: pd.DataFrame, cols_nombres: List[str]) -> Set[str]:
        terminos_clave: Set[str] = set()
        if df_coincidencias is None or df_coincidencias.empty or not cols_nombres:
            return terminos_clave

        cols_validas = [c for c in cols_nombres if c in df_coincidencias.columns]
        if not cols_validas:
            logger.warning("Columnas de diccionario no existen en coincidencias.")
            return terminos_clave

        for col_nombre in cols_validas:
            try:
                for texto_celda in df_coincidencias[col_nombre].dropna().astype(str):
                    palabras = self.extractor_magnitud._normalizar_texto(texto_celda).split()
                    terminos_clave.update(p for p in palabras if len(p) > 2 and p.isalnum())
            except Exception as e:
                logger.warning(f"Error extrayendo términos de col '{col_nombre}': {e}")

        logger.info(f"{len(terminos_clave)} términos extraídos del diccionario para búsqueda secundaria.")
        if terminos_clave: logger.debug(f"Términos clave (muestra): {list(terminos_clave)[:10]}...")
        return terminos_clave

---
## 5. `interfaz_grafica.py`

In [ ]:
# interfaz_grafica.py
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import pandas as pd
import platform
import json
import os
from pathlib import Path
import logging
from typing import Optional, List, Dict, Any

from motor_busqueda import MotorBusqueda
from enums import OrigenResultados
from config import CONFIG_FILE_NAME # Importar el nombre del archivo de configuración

logger = logging.getLogger(__name__)

class InterfazGrafica(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Buscador Avanzado (v0.7.1 - Mapeo Magnitudes)")
        self.geometry("1250x800") # Considerar mover a config.py

        self.config = self._cargar_configuracion()
        indices_cfg = self.config.get("indices_columnas_busqueda_dic", [])
        self.motor = MotorBusqueda(indices_diccionario_cfg=indices_cfg)

        self.resultados_actuales: Optional[pd.DataFrame] = None
        self.texto_busqueda_var = tk.StringVar(self)
        self.texto_busqueda_var.trace_add("write", self._on_texto_busqueda_change)
        self.ultimo_termino_buscado: Optional[str] = None
        self.reglas_guardadas: List[Dict[str, Any]] = []

        self.fcds_de_ultima_busqueda: Optional[pd.DataFrame] = None
        self.desc_finales_de_ultima_busqueda: Optional[pd.DataFrame] = None

        self.origen_principal_resultados: OrigenResultados = OrigenResultados.NINGUNO
        self.color_fila_par = "white"; self.color_fila_impar = "#f0f0f0"
        self.op_buttons: Dict[str, ttk.Button] = {}

        self._configurar_estilo_ttk()
        self._crear_widgets()
        self._configurar_grid()
        self._configurar_eventos()
        self._configurar_tags_treeview()
        self._configurar_orden_tabla(self.tabla_resultados)
        self._configurar_orden_tabla(self.tabla_diccionario)

        self._actualizar_estado("Listo. Cargue Diccionario y Descripciones.")
        self._deshabilitar_botones_operadores()
        self._actualizar_botones_estado_general()
        logger.info("Interfaz Gráfica (v0.7.1 - Mapeo) inicializada.")

    def _on_texto_busqueda_change(self, var_name: str, index: str, mode: str):
        self._actualizar_estado_botones_operadores()

    def _cargar_configuracion(self) -> Dict:
        config = {}
        if os.path.exists(CONFIG_FILE_NAME): # Usar constante de config.py
            try:
                with open(CONFIG_FILE_NAME, 'r', encoding='utf-8') as f: config = json.load(f)
                logger.info(f"Configuración cargada desde: {CONFIG_FILE_NAME}")
            except Exception as e:
                logger.error(f"Error al cargar config: {e}")
        else:
            logger.info(f"Archivo de configuración '{CONFIG_FILE_NAME}' no encontrado. Se creará al cerrar.")

        last_dic_path_str = config.get("last_dic_path")
        config["last_dic_path"] = str(Path(last_dic_path_str)) if last_dic_path_str else None
        last_desc_path_str = config.get("last_desc_path")
        config["last_desc_path"] = str(Path(last_desc_path_str)) if last_desc_path_str else None
        config.setdefault("indices_columnas_busqueda_dic", [])
        return config

    def _guardar_configuracion(self):
        self.config["last_dic_path"] = str(self.motor.archivo_diccionario_actual) if self.motor.archivo_diccionario_actual else None
        self.config["last_desc_path"] = str(self.motor.archivo_descripcion_actual) if self.motor.archivo_descripcion_actual else None
        self.config["indices_columnas_busqueda_dic"] = self.motor.indices_columnas_busqueda_dic
        try:
            with open(CONFIG_FILE_NAME, 'w', encoding='utf-8') as f: # Usar constante de config.py
                json.dump(self.config, f, indent=4)
            logger.info(f"Configuración guardada en: {CONFIG_FILE_NAME}")
        except Exception as e:
            logger.error(f"Error al guardar config: {e}")
            messagebox.showerror("Error Configuración", f"No se pudo guardar config:\n{e}")

    def _configurar_estilo_ttk(self):
        style = ttk.Style(self); themes = style.theme_names(); os_name = platform.system()
        prefs = {"Windows":["vista","xpnative","clam"],"Darwin":["aqua","clam"],"Linux":["clam","alt","default"]}
        theme_to_use = next((t for t in prefs.get(os_name, ["clam","default"]) if t in themes), None)
        if not theme_to_use:
            theme_to_use = style.theme_use() if style.theme_use() else ("default" if "default" in themes else (themes[0] if themes else None))
        if theme_to_use:
            logger.info(f"Aplicando tema TTK: {theme_to_use}")
            try:
                style.theme_use(theme_to_use)
                style.configure("Operator.TButton", padding=(2, 1), font=('TkDefaultFont', 9))
            except tk.TclError as e: logger.warning(f"No se pudo aplicar tema '{theme_to_use}': {e}.")
        else: logger.warning("No se encontró tema TTK disponible.")

    def _crear_widgets(self):
        self.marco_controles = ttk.LabelFrame(self, text="Controles")
        self.btn_cargar_diccionario = ttk.Button(self.marco_controles, text="Cargar Diccionario", command=self._cargar_diccionario)
        self.lbl_dic_cargado = ttk.Label(self.marco_controles, text="Dic: Ninguno", width=20, anchor=tk.W, relief=tk.SUNKEN, borderwidth=1)
        self.btn_cargar_descripciones = ttk.Button(self.marco_controles, text="Cargar Descripciones", command=self._cargar_excel_descripcion)
        self.lbl_desc_cargado = ttk.Label(self.marco_controles, text="Desc: Ninguno", width=20, anchor=tk.W, relief=tk.SUNKEN, borderwidth=1)

        self.frame_ops = ttk.Frame(self.marco_controles)
        op_buttons_defs = [
            ("+", "+"), ("|", "|"), ("#", "#"), (">", ">"),
            ("<", "<"), ("≥", ">="), ("≤", "<="), ("-", "-")
        ]
        for i, (text, op_val) in enumerate(op_buttons_defs):
            btn = ttk.Button(
                self.frame_ops, text=text,
                command=lambda op=op_val: self._insertar_operador_validado(op),
                style="Operator.TButton", width=3
            )
            btn.grid(row=0, column=i, padx=1, pady=1, sticky="nsew")
            self.op_buttons[op_val] = btn

        self.entrada_busqueda = ttk.Entry(self.marco_controles, width=50, textvariable=self.texto_busqueda_var)
        self.btn_buscar = ttk.Button(self.marco_controles, text="Buscar", command=self._ejecutar_busqueda)
        self.btn_salvar_regla = ttk.Button(self.marco_controles, text="Salvar Regla", command=self._salvar_regla_actual)
        self.btn_ayuda = ttk.Button(self.marco_controles, text="?", command=self._mostrar_ayuda, width=3)
        self.btn_exportar = ttk.Button(self.marco_controles, text="Exportar", command=self._exportar_resultados)

        self.lbl_tabla_diccionario = ttk.Label(self, text="Vista Previa Diccionario:")
        self.lbl_tabla_resultados = ttk.Label(self, text="Resultados / Descripciones:")

        self.frame_tabla_diccionario = ttk.Frame(self)
        self.tabla_diccionario = ttk.Treeview(self.frame_tabla_diccionario, show="headings", height=8)
        self.scrolly_diccionario = ttk.Scrollbar(self.frame_tabla_diccionario, orient="vertical", command=self.tabla_diccionario.yview)
        self.scrollx_diccionario = ttk.Scrollbar(self.frame_tabla_diccionario, orient="horizontal", command=self.tabla_diccionario.xview)
        self.tabla_diccionario.configure(yscrollcommand=self.scrolly_diccionario.set, xscrollcommand=self.scrollx_diccionario.set)

        self.frame_tabla_resultados = ttk.Frame(self)
        self.tabla_resultados = ttk.Treeview(self.frame_tabla_resultados, show="headings")
        self.scrolly_resultados = ttk.Scrollbar(self.frame_tabla_resultados, orient="vertical", command=self.tabla_resultados.yview)
        self.scrollx_resultados = ttk.Scrollbar(self.frame_tabla_resultados, orient="horizontal", command=self.tabla_resultados.xview)
        self.tabla_resultados.configure(yscrollcommand=self.scrolly_resultados.set, xscrollcommand=self.scrollx_resultados.set)

        self.barra_estado = ttk.Label(self, text="", relief=tk.SUNKEN, anchor=tk.W, borderwidth=1)
        self._actualizar_etiquetas_archivos()

    def _configurar_grid(self):
        self.grid_rowconfigure(2, weight=1); self.grid_rowconfigure(4, weight=3)
        self.grid_columnconfigure(0, weight=1)
        self.marco_controles.grid(row=0, column=0, sticky="new", padx=10, pady=(10, 5))
        self.marco_controles.grid_columnconfigure(1, weight=1)
        self.marco_controles.grid_columnconfigure(3, weight=1)

        self.btn_cargar_diccionario.grid(row=0, column=0, padx=(5,0), pady=5, sticky="w")
        self.lbl_dic_cargado.grid(row=0, column=1, padx=(2,10), pady=5, sticky="ew")
        self.btn_cargar_descripciones.grid(row=0, column=2, padx=(5,0), pady=5, sticky="w")
        self.lbl_desc_cargado.grid(row=0, column=3, padx=(2,5), pady=5, sticky="ew")

        self.frame_ops.grid(row=1, column=0, columnspan=4, padx=5, pady=(5,0), sticky="ew")
        for i in range(len(self.op_buttons)): self.frame_ops.grid_columnconfigure(i, weight=1)

        self.entrada_busqueda.grid(row=2, column=0, columnspan=2, padx=5, pady=(0,5), sticky="ew")
        self.btn_buscar.grid(row=2, column=2, padx=(2,0), pady=(0,5), sticky="w")
        self.btn_salvar_regla.grid(row=2, column=3, padx=(2,0), pady=(0,5), sticky="w")
        self.btn_ayuda.grid(row=2, column=4, padx=(2,0), pady=(0,5), sticky="w")
        self.btn_exportar.grid(row=2, column=5, padx=(10, 5), pady=(0,5), sticky="e")

        self.lbl_tabla_diccionario.grid(row=1, column=0, sticky="sw", padx=10, pady=(10, 0))
        self.frame_tabla_diccionario.grid(row=2, column=0, sticky="nsew", padx=10, pady=(0, 10))
        self.frame_tabla_diccionario.grid_rowconfigure(0, weight=1); self.frame_tabla_diccionario.grid_columnconfigure(0, weight=1)
        self.tabla_diccionario.grid(row=0, column=0, sticky="nsew"); self.scrolly_diccionario.grid(row=0, column=1, sticky="ns"); self.scrollx_diccionario.grid(row=1, column=0, sticky="ew")

        self.lbl_tabla_resultados.grid(row=3, column=0, sticky="sw", padx=10, pady=(0, 0))
        self.frame_tabla_resultados.grid(row=4, column=0, sticky="nsew", padx=10, pady=(0, 10))
        self.frame_tabla_resultados.grid_rowconfigure(0, weight=1); self.frame_tabla_resultados.grid_columnconfigure(0, weight=1)
        self.tabla_resultados.grid(row=0, column=0, sticky="nsew"); self.scrolly_resultados.grid(row=0, column=1, sticky="ns"); self.scrollx_resultados.grid(row=1, column=0, sticky="ew")

        self.barra_estado.grid(row=5, column=0, sticky="sew", padx=0, pady=(5, 0))

    def _configurar_eventos(self):
        self.entrada_busqueda.bind("<Return>", lambda event: self._ejecutar_busqueda())
        self.protocol("WM_DELETE_WINDOW", self.on_closing)

    def _actualizar_estado(self, mensaje: str):
        self.barra_estado.config(text=mensaje)
        logger.info(f"Estado UI: {mensaje}")
        self.update_idletasks()

    def _mostrar_ayuda(self):
        # El texto de ayuda permanece igual, ya que la sintaxis de búsqueda para el usuario no ha cambiado.
        ayuda = """Sintaxis de Búsqueda:
-------------------------------------
- Texto simple: Busca la palabra o frase (insensible a mayús/minús y acentos). Ej: `router cisco`
- Operadores Lógicos:
  * `término1 + término2`: Busca filas con AMBOS (AND). Ej: `tarjeta + 16 puertos`
  * `término1 | término2` (o `/`): Busca filas con AL MENOS UNO (OR). Ej: `modulo | SFP`
- Comparaciones numéricas (unidad opcional, si se usa debe coincidir con mapeo interno):
  * `>num[UNIDAD]`: Mayor. Ej: `>1000` o `>1000w`
  * `<num[UNIDAD]`: Menor. Ej: `<50` o `<50v`
  * `>=num[UNIDAD]` o `≥num[UNIDAD]`: Mayor o igual. Ej: `>=48a`
  * `<=num[UNIDAD]` o `≤num[UNIDAD]`: Menor o igual. Ej: `<=10.5w`
- Rangos numéricos (unidad opcional, ambos extremos incluidos):
  * `num1-num2[UNIDAD]`: Entre num1 y num2. Ej: `10-20` o `50-100V`
- Negación (excluir):
  * `#término`: `término` puede ser texto, comparación o rango.
    Ej: `switch + #gestionable`
    Ej: `tarjeta + #>8 puertos`

Modo de Búsqueda:
1. El término se busca primero en el Diccionario.
2. Si hay coincidencias (FCDs), se extraen textos y se buscan en Descripciones.
3. Si no, se preguntará para buscar el término original directamente en Descripciones.
4. Búsqueda vacía: Muestra todas las descripciones.
"""
        messagebox.showinfo("Ayuda - Sintaxis de Búsqueda", ayuda)

    def _configurar_tags_treeview(self):
        for tabla in [self.tabla_diccionario, self.tabla_resultados]:
            tabla.tag_configure('par', background=self.color_fila_par)
            tabla.tag_configure('impar', background=self.color_fila_impar)

    def _configurar_orden_tabla(self, tabla: ttk.Treeview):
        cols = tabla["columns"]
        if cols:
            for col in cols:
                tabla.heading(col, text=str(col), anchor=tk.W,
                              command=lambda c=col, t=tabla: self._ordenar_columna(t, c, False))

    def _ordenar_columna(self, tabla: ttk.Treeview, col: str, reverse: bool):
        df_para_ordenar = None
        if tabla == self.tabla_diccionario:
            df_para_ordenar = self.motor.datos_diccionario
        elif tabla == self.tabla_resultados:
            df_para_ordenar = self.resultados_actuales

        if df_para_ordenar is None or df_para_ordenar.empty or col not in df_para_ordenar.columns:
            logging.debug(f"No se puede ordenar tabla por columna '{col}'.")
            tabla.heading(col, command=lambda c=col, t=tabla: self._ordenar_columna(t, c, not reverse))
            return

        logging.info(f"Ordenando tabla por columna '{col}', descendente={reverse}")
        try:
            col_to_sort_by = df_para_ordenar[col]
            try: # Intento de orden numérico inteligente
                nan_mask = pd.to_numeric(col_to_sort_by, errors='coerce').isna()
                numeric_part = col_to_sort_by[~nan_mask]
                nan_part = col_to_sort_by[nan_mask]

                if not numeric_part.empty:
                    temp_numeric = pd.to_numeric(numeric_part, errors='coerce')
                    if not temp_numeric.isna().all():
                        sorted_numeric_indices = temp_numeric.sort_values(ascending=not reverse).index
                        final_order_indices = sorted_numeric_indices.tolist() + nan_part.index.tolist()
                        df_ordenado = df_para_ordenar.loc[final_order_indices]
                    else: raise ValueError("No convertible a numérico fiable")
                else:
                    df_ordenado = df_para_ordenar.sort_values(by=col, ascending=not reverse, na_position='last', key=lambda x: x.astype(str).str.lower())
            except (ValueError, TypeError): # Fallback a ordenación de texto
                df_ordenado = df_para_ordenar.sort_values(
                    by=col, ascending=not reverse, na_position='last',
                    key=lambda x: x.astype(str).str.lower() if pd.api.types.is_string_dtype(x) or pd.api.types.is_object_dtype(x) else x
                )

            if tabla == self.tabla_diccionario:
                self.motor.datos_diccionario = df_ordenado
                self._actualizar_tabla(tabla, df_ordenado, limite_filas=100)
            elif tabla == self.tabla_resultados:
                self.resultados_actuales = df_ordenado
                self._actualizar_tabla(tabla, df_ordenado)

            tabla.heading(col, command=lambda c=col, t=tabla: self._ordenar_columna(t, c, not reverse))
            self._actualizar_estado(f"Tabla ordenada por '{col}' ({'Asc' if not reverse else 'Desc'}).")
        except Exception as e:
            logging.exception(f"Error ordenando por columna '{col}'")
            messagebox.showerror("Error al Ordenar", f"No se pudo ordenar por '{col}':\n{e}")
            tabla.heading(col, command=lambda c=col, t=tabla: self._ordenar_columna(t, c, False))


    def _actualizar_tabla(self, tabla: ttk.Treeview, datos: Optional[pd.DataFrame], limite_filas: Optional[int] = None, columnas_a_mostrar: Optional[List[str]] = None):
        is_diccionario = tabla == self.tabla_diccionario
        logger.debug(f"Actualizando tabla {'Diccionario' if is_diccionario else 'Resultados'}.")
        try:
            for i in tabla.get_children(): tabla.delete(i)
        except tk.TclError as e: logger.warning(f"Error Tcl limpiando tabla: {e}"); pass
        tabla["columns"] = ()

        if datos is None or datos.empty:
            logger.debug("Sin datos para mostrar en tabla.")
            self._configurar_orden_tabla(tabla)
            return

        cols_df = list(datos.columns)
        cols_finales = [c for c in (columnas_a_mostrar or cols_df) if c in cols_df] or cols_df

        if not cols_finales:
            logger.warning("DataFrame sin columnas o columnas seleccionadas no existen.")
            self._configurar_orden_tabla(tabla)
            return

        df_vista = datos[cols_finales]
        tabla["columns"] = tuple(cols_finales)

        for col in cols_finales:
            tabla.heading(col, text=str(col), anchor=tk.W)
            try:
                col_str = df_vista[col].astype(str)
                ancho_cont = col_str.str.len().max() if not col_str.empty else 0
                ancho_cab = len(str(col))
                ancho = max(70, min(int(max(ancho_cab * 8, ancho_cont * 6.5) + 25), 400))
                tabla.column(col, anchor=tk.W, width=ancho, minwidth=70)
            except Exception:
                tabla.column(col, anchor=tk.W, width=100, minwidth=50)

        num_iterar = limite_filas if is_diccionario and limite_filas is not None else len(df_vista)
        df_iterar = df_vista.head(num_iterar)

        for i, (_, row) in enumerate(df_iterar.iterrows()):
            vals = [str(v) if pd.notna(v) else "" for v in row.values]
            tag = 'par' if i % 2 == 0 else 'impar'
            try:
                tabla.insert("", "end", values=vals, tags=(tag,))
            except tk.TclError:
                try:
                    vals_ascii = [v.encode('ascii', 'ignore').decode('ascii') for v in vals]
                    tabla.insert("", "end", values=vals_ascii, tags=(tag,))
                except Exception as e_inner:
                    logger.error(f"Fallo fallback ASCII fila {i}: {e_inner}")

        self._configurar_orden_tabla(tabla)

    def _actualizar_etiquetas_archivos(self):
        dic_p = self.motor.archivo_diccionario_actual
        desc_p = self.motor.archivo_descripcion_actual
        dic_n = dic_p.name if dic_p else "Ninguno"
        desc_n = desc_p.name if desc_p else "Ninguno"

        max_l = 25
        dic_d = f"Dic: {dic_n}" if len(dic_n) <= max_l else f"Dic: ...{dic_n[-(max_l-4):]}"
        desc_d = f"Desc: {desc_n}" if len(desc_n) <= max_l else f"Desc: ...{desc_n[-(max_l-4):]}"

        self.lbl_dic_cargado.config(text=dic_d, foreground="green" if dic_p else "red")
        self.lbl_desc_cargado.config(text=desc_d, foreground="green" if desc_p else "red")

    def _actualizar_botones_estado_general(self):
        dic_ok = self.motor.datos_diccionario is not None
        desc_ok = self.motor.datos_descripcion is not None

        if dic_ok or desc_ok: self._actualizar_estado_botones_operadores()
        else: self._deshabilitar_botones_operadores()

        self.btn_buscar['state'] = 'normal' if dic_ok and desc_ok else 'disabled'

        salvar_ok = False
        if self.ultimo_termino_buscado and self.origen_principal_resultados != OrigenResultados.NINGUNO:
            if self.origen_principal_resultados.es_via_diccionario:
                if (self.fcds_de_ultima_busqueda is not None and not self.fcds_de_ultima_busqueda.empty) or \
                   (self.desc_finales_de_ultima_busqueda is not None and not self.desc_finales_de_ultima_busqueda.empty and \
                    self.origen_principal_resultados == OrigenResultados.VIA_DICCIONARIO_CON_RESULTADOS_DESC):
                    salvar_ok = True
            elif self.origen_principal_resultados.es_directo_descripcion or \
                 self.origen_principal_resultados == OrigenResultados.DIRECTO_DESCRIPCION_VACIA:
                if self.desc_finales_de_ultima_busqueda is not None: salvar_ok = True

        self.btn_salvar_regla['state'] = 'normal' if salvar_ok else 'disabled'
        self.btn_exportar['state'] = 'normal' if self.reglas_guardadas else 'disabled'

    def _cargar_diccionario(self):
        last_dir = str(Path(self.config.get("last_dic_path","")).parent) if self.config.get("last_dic_path") and Path(self.config.get("last_dic_path","")).exists() else os.getcwd()
        ruta = filedialog.askopenfilename(title="Cargar Diccionario", filetypes=[("Excel", "*.xlsx *.xls")], initialdir=last_dir)
        if not ruta: return

        self._actualizar_estado(f"Cargando diccionario: {Path(ruta).name}...")
        self._actualizar_tabla(self.tabla_diccionario, None); self._actualizar_tabla(self.tabla_resultados, None)
        self.resultados_actuales = self.fcds_de_ultima_busqueda = self.desc_finales_de_ultima_busqueda = None
        self.origen_principal_resultados = OrigenResultados.NINGUNO

        exito, msg = self.motor.cargar_excel_diccionario(ruta)
        if exito:
            self.config["last_dic_path"] = ruta; self._guardar_configuracion()
            df_dic = self.motor.datos_diccionario
            if df_dic is not None:
                n_filas = len(df_dic)
                cols_b, _ = self.motor._obtener_nombres_columnas_busqueda_df(df_dic, self.motor.indices_columnas_busqueda_dic, "dic (preview)")
                idxs_str = ', '.join(map(str, self.motor.indices_columnas_busqueda_dic or [])) if self.motor.indices_columnas_busqueda_dic and self.motor.indices_columnas_busqueda_dic != [-1] else "Todas Texto"
                lbl_txt = f"Vista Previa Dic ({n_filas} filas)" + (f" (Cols: {', '.join(cols_b)} - Idx: {idxs_str})" if cols_b else "")
                self.lbl_tabla_diccionario.config(text=lbl_txt)
                self._actualizar_tabla(self.tabla_diccionario, df_dic, limite_filas=100, columnas_a_mostrar=cols_b)
                self.title(f"Buscador - Dic: {Path(ruta).name}")
                self._actualizar_estado(f"Diccionario '{Path(ruta).name}' ({n_filas} filas) cargado.")
        else:
            self._actualizar_estado(f"Error cargando diccionario: {msg or 'Desconocido'}")
            if msg: messagebox.showerror("Error Carga Diccionario", msg)
            self.title("Buscador Avanzado (v0.7.1 - Mapeo Magnitudes)")
        self._actualizar_etiquetas_archivos()
        self._actualizar_botones_estado_general()

    def _cargar_excel_descripcion(self):
        last_dir = str(Path(self.config.get("last_desc_path","")).parent) if self.config.get("last_desc_path") and Path(self.config.get("last_desc_path","")).exists() else os.getcwd()
        ruta = filedialog.askopenfilename(title="Cargar Descripciones", filetypes=[("Excel", "*.xlsx *.xls")], initialdir=last_dir)
        if not ruta: return

        self._actualizar_estado(f"Cargando descripciones: {Path(ruta).name}...")
        self.resultados_actuales = self.desc_finales_de_ultima_busqueda = None
        self.origen_principal_resultados = OrigenResultados.NINGUNO
        self._actualizar_tabla(self.tabla_resultados, None)

        exito, msg = self.motor.cargar_excel_descripcion(ruta)
        if exito:
            self.config["last_desc_path"] = ruta; self._guardar_configuracion()
            df_desc = self.motor.datos_descripcion
            if df_desc is not None:
                n_filas = len(df_desc)
                self._actualizar_estado(f"Descripciones '{Path(ruta).name}' ({n_filas} filas) cargadas.")
                self._actualizar_tabla(self.tabla_resultados, df_desc)
                dic_title = Path(self.motor.archivo_diccionario_actual or "").name or "N/A"
                self.title(f"Buscador - Dic: {dic_title} | Desc: {Path(ruta).name}")
        else:
            self._actualizar_estado(f"Error cargando descripciones: {msg or 'Desconocido'}")
            if msg: messagebox.showerror("Error Carga Descripciones", msg)
            dic_title = Path(self.motor.archivo_diccionario_actual or "").name or "N/A"
            self.title(f"Buscador - Dic: {dic_title} | Desc: N/A" if self.motor.archivo_diccionario_actual else "Buscador Avanzado (v0.7.1)")
        self._actualizar_etiquetas_archivos()
        self._actualizar_botones_estado_general()

    def _ejecutar_busqueda(self):
        if not self.motor.datos_diccionario or not self.motor.datos_descripcion:
            messagebox.showwarning("Archivos Faltantes", "Cargue Diccionario y Descripciones.")
            return

        term = self.texto_busqueda_var.get()
        self.ultimo_termino_buscado = term
        self.resultados_actuales = self.fcds_de_ultima_busqueda = self.desc_finales_de_ultima_busqueda = None
        self.origen_principal_resultados = OrigenResultados.NINGUNO
        self._actualizar_tabla(self.tabla_resultados, None)
        self._actualizar_estado(f"Buscando '{term}'...")

        res_df, origen, fcds, err_msg = self.motor.buscar(term, True)
        self.fcds_de_ultima_busqueda = fcds; self.origen_principal_resultados = origen
        df_desc_cols = self.motor.datos_descripcion.columns if self.motor.datos_descripcion is not None else []

        if err_msg and (origen.es_error_operacional or origen.es_termino_invalido) : # Error crítico del motor
             messagebox.showerror("Error de Búsqueda (Motor)", f"Error: {err_msg}")
             self._actualizar_estado(f"Error en motor: {err_msg}")
             self.resultados_actuales = pd.DataFrame(columns=df_desc_cols)
        elif origen.es_error_carga or origen.es_error_configuracion :
             msg_sh = err_msg or f"Error: {origen.name}"
             messagebox.showerror("Error de Búsqueda", msg_sh)
             self._actualizar_estado(msg_sh)
             self.resultados_actuales = pd.DataFrame(columns=df_desc_cols)
        elif origen == OrigenResultados.VIA_DICCIONARIO_CON_RESULTADOS_DESC:
            self.resultados_actuales = res_df
            self._actualizar_estado(f"'{term}': {len(fcds or [])} en Dic, {len(res_df or [])} en Desc.")
        elif origen in [OrigenResultados.VIA_DICCIONARIO_SIN_RESULTADOS_DESC, OrigenResultados.VIA_DICCIONARIO_SIN_TERMINOS_VALIDOS] or \
             (fcds is not None and fcds.empty and origen == OrigenResultados.VIA_DICCIONARIO_SIN_RESULTADOS_DESC):
            self.resultados_actuales = res_df # DF vacío
            msg_fcd_info = f"{len(fcds or [])} coincidencias en Diccionario" if fcds is not None and not fcds.empty else "Ninguna coincidencia en Diccionario"
            msg_desc_issue = "pero no se extrajeron términos para Desc." if origen == OrigenResultados.VIA_DICCIONARIO_SIN_TERMINOS_VALIDOS else "sin resultados en Desc."
            self._actualizar_estado(f"'{term}': {msg_fcd_info}, {msg_desc_issue.split('.')[0]}.")
            if messagebox.askyesno("Búsqueda Alternativa", f"{msg_fcd_info} para '{term}', {msg_desc_issue}\n\nBuscar '{term}' directo en Descripciones?"):
                self._actualizar_estado(f"Buscando directo '{term}' en descripciones...")
                res_df_dir, origen_dir, _, err_msg_dir = self.motor.buscar(term, False)
                if err_msg_dir and (origen_dir.es_error_operacional or origen_dir.es_termino_invalido):
                     messagebox.showerror("Error Búsqueda Directa", f"Error: {err_msg_dir}")
                     self._actualizar_estado(f"Error búsqueda directa: {err_msg_dir}")
                elif origen_dir.es_error_carga or origen_dir.es_error_configuracion:
                     msg_sh_dir = err_msg_dir or f"Error en búsqueda directa: {origen_dir.name}"
                     messagebox.showerror("Error Búsqueda Directa", msg_sh_dir)
                     self._actualizar_estado(msg_sh_dir)
                else:
                    self.resultados_actuales = res_df_dir; self.origen_principal_resultados = origen_dir
                    self.fcds_de_ultima_busqueda = None
                    num_rdd = len(self.resultados_actuales or [])
                    self._actualizar_estado(f"Búsqueda directa '{term}': {num_rdd} resultados.")
                    if num_rdd == 0 and origen_dir == OrigenResultados.DIRECTO_DESCRIPCION_VACIA and term.strip():
                        messagebox.showinfo("Información", f"No se encontraron resultados para '{term}' directo.")
        elif origen == OrigenResultados.DIRECTO_DESCRIPCION_CON_RESULTADOS :
            self.resultados_actuales = res_df
            self._actualizar_estado(f"Búsqueda directa '{term}': {len(res_df or [])} resultados.")
        elif origen == OrigenResultados.DIRECTO_DESCRIPCION_VACIA:
            self.resultados_actuales = res_df
            self._actualizar_estado(f"Búsqueda directa '{term}': 0 resultados.")
            if term.strip(): messagebox.showinfo("Información", f"No se encontraron resultados para '{term}' directo.")

        if self.resultados_actuales is None: self.resultados_actuales = pd.DataFrame(columns=df_desc_cols)
        self.desc_finales_de_ultima_busqueda = self.resultados_actuales.copy()
        self._actualizar_tabla(self.tabla_resultados, self.resultados_actuales)
        self._actualizar_botones_estado_general()
        if self.motor.datos_diccionario is not None and not self.motor.datos_diccionario.empty:
            self._buscar_y_enfocar_en_preview()

    def _buscar_y_enfocar_en_preview(self): # Simplificado para brevedad, la lógica es de UI
        df_dic = self.motor.datos_diccionario
        if df_dic is None or df_dic.empty: return
        term_raw = self.texto_busqueda_var.get()
        if not term_raw.strip(): return

        op_n1, segs_n1 = self.motor._parsear_nivel1_or(term_raw)
        if not segs_n1: return
        op_n2, terms_n2 = self.motor._parsear_nivel2_and(segs_n1[0])
        if not terms_n2: return
        term_focus_raw = terms_n2[0][1:].strip() if terms_n2[0].startswith("#") else terms_n2[0].strip()
        if not term_focus_raw: return
        term_focus_norm = self.motor.extractor_magnitud._normalizar_texto(term_focus_raw)
        if not term_focus_norm: return

        items_ids = self.tabla_diccionario.get_children('')
        if not items_ids: return

        logger.info(f"Enfocando '{term_focus_norm}' en preview diccionario...")
        found_id = None
        cols_preview = self.tabla_diccionario["columns"] or []
        # La lógica original para encontrar el item_id era compleja y dependía de la estructura de datos
        # en la preview. Aquí una simplificación conceptual:
        for item_id in items_ids:
            try:
                vals = self.tabla_diccionario.item(item_id, 'values')
                if any(term_focus_norm in self.motor.extractor_magnitud._normalizar_texto(str(v)) for v in vals if v):
                    found_id = item_id; break
            except Exception: continue

        if found_id:
            logger.info(f"Término '{term_focus_norm}' enfocado (ID: {found_id}).")
            try:
                if self.tabla_diccionario.selection(): self.tabla_diccionario.selection_remove(self.tabla_diccionario.selection())
                self.tabla_diccionario.selection_set(found_id); self.tabla_diccionario.see(found_id); self.tabla_diccionario.focus(found_id)
            except Exception as e: logger.error(f"Error enfocando item {found_id}: {e}")
        else: logger.info(f"Término '{term_focus_norm}' no enfocado en preview.")


    def _salvar_regla_actual(self): # La lógica interna de esta función es extensa y se mantiene
        origen_nombre = self.origen_principal_resultados.name
        logger.info(f"Salvando regla. Origen: {origen_nombre}, Último término: '{self.ultimo_termino_buscado}'")

        if not self.ultimo_termino_buscado and not (self.origen_principal_resultados == OrigenResultados.DIRECTO_DESCRIPCION_VACIA and self.desc_finales_de_ultima_busqueda is not None):
            messagebox.showerror("Error", "No hay término de búsqueda o resultados válidos."); return

        term_orig_regla = self.ultimo_termino_buscado or ''
        op_n1, segs_n1 = self.motor._parsear_nivel1_or(term_orig_regla)
        parsed_segs = []
        for seg in segs_n1:
            op_n2, terms_n2 = self.motor._parsear_nivel2_and(seg)
            parsed_segs.append({"operador_segmento_and": op_n2, "terminos_analizados": self.motor._analizar_terminos(terms_n2)})

        regla_base = {'termino_busqueda_original': term_orig_regla, 'operador_principal_or': op_n1,
                      'segmentos_parseados_para_and': parsed_segs, 'fuente_original_guardado': origen_nombre,
                      'timestamp': pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")}
        salvo = False

        if self.origen_principal_resultados.es_via_diccionario:
            decision = self._mostrar_dialogo_seleccion_salvado_via_diccionario()
            if decision['confirmed']:
                if decision['save_fcd'] and self.fcds_de_ultima_busqueda is not None and not self.fcds_de_ultima_busqueda.empty:
                    self.reglas_guardadas.append({**regla_base, 'tipo_datos_guardados': "COINCIDENCIAS_DICCIONARIO", 'datos_snapshot': self.fcds_de_ultima_busqueda.to_dict('records')}); salvo = True
                if decision['save_rfd'] and self.desc_finales_de_ultima_busqueda is not None and not self.desc_finales_de_ultima_busqueda.empty and self.origen_principal_resultados == OrigenResultados.VIA_DICCIONARIO_CON_RESULTADOS_DESC:
                    self.reglas_guardadas.append({**regla_base, 'tipo_datos_guardados': "RESULTADOS_DESCRIPCION_VIA_DICCIONARIO", 'datos_snapshot': self.desc_finales_de_ultima_busqueda.to_dict('records')}); salvo = True
        elif self.origen_principal_resultados.es_directo_descripcion or self.origen_principal_resultados == OrigenResultados.DIRECTO_DESCRIPCION_VACIA:
             if self.desc_finales_de_ultima_busqueda is not None:
                tipo = "TODAS_LAS_DESCRIPCIONES" if self.origen_principal_resultados == OrigenResultados.DIRECTO_DESCRIPCION_VACIA and not term_orig_regla.strip() else "RESULTADOS_DESCRIPCION_DIRECTA"
                self.reglas_guardadas.append({**regla_base, 'tipo_datos_guardados': tipo, 'datos_snapshot': self.desc_finales_de_ultima_busqueda.to_dict('records')}); salvo = True
             else: messagebox.showwarning("Sin Datos", "No hay resultados de descripción para salvar.")
        else:
            if self.origen_principal_resultados != OrigenResultados.NINGUNO and not (self.origen_principal_resultados.es_error_carga or self.origen_principal_resultados.es_error_configuracion or self.origen_principal_resultados.es_error_operacional or self.origen_principal_resultados.es_termino_invalido):
                messagebox.showerror("Error", f"No se puede determinar qué salvar para origen: {origen_nombre}.")
            else: messagebox.showwarning("Nada que Salvar", "No hay resultados válidos para salvar.")

        if salvo: self._actualizar_estado(f"Regla(s) guardada(s). Total: {len(self.reglas_guardadas)}.")
        elif self.ultimo_termino_buscado or self.origen_principal_resultados == OrigenResultados.DIRECTO_DESCRIPCION_VACIA :
            self._actualizar_estado("Ninguna regla salvada.")
        self._actualizar_botones_estado_general()


    def _mostrar_dialogo_seleccion_salvado_via_diccionario(self) -> Dict[str, bool]: # UI, se mantiene
        decision = {'confirmed': False, 'save_fcd': False, 'save_rfd': False}
        dialog = tk.Toplevel(self); dialog.title("Seleccionar Datos a Salvar"); dialog.geometry("400x200")
        dialog.resizable(False, False); dialog.transient(self); dialog.grab_set()

        var_fcd = tk.BooleanVar(value=(self.fcds_de_ultima_busqueda is not None and not self.fcds_de_ultima_busqueda.empty))
        var_rfd = tk.BooleanVar(value=(self.desc_finales_de_ultima_busqueda is not None and not self.desc_finales_de_ultima_busqueda.empty and self.origen_principal_resultados == OrigenResultados.VIA_DICCIONARIO_CON_RESULTADOS_DESC))

        ttk.Label(dialog, text="¿Qué datos salvar de la búsqueda vía Diccionario?").pack(pady=10, padx=10)
        chk_fcd = ttk.Checkbutton(dialog, text="Coincidencias del Diccionario (FCDs)", variable=var_fcd)
        chk_fcd.pack(anchor=tk.W, padx=20)
        chk_fcd['state'] = 'normal' if self.fcds_de_ultima_busqueda is not None and not self.fcds_de_ultima_busqueda.empty else 'disabled'
        chk_rfd = ttk.Checkbutton(dialog, text="Resultados Finales de Descripciones (RFDs)", variable=var_rfd)
        chk_rfd.pack(anchor=tk.W, padx=20)
        chk_rfd['state'] = 'normal' if self.desc_finales_de_ultima_busqueda is not None and not self.desc_finales_de_ultima_busqueda.empty and self.origen_principal_resultados == OrigenResultados.VIA_DICCIONARIO_CON_RESULTADOS_DESC else 'disabled'

        frm_btns = ttk.Frame(dialog); frm_btns.pack(pady=15)
        def on_ok():
            decision.update({'confirmed': True, 'save_fcd': var_fcd.get(), 'save_rfd': var_rfd.get()})
            if not decision['save_fcd'] and not decision['save_rfd']:
                fcd_ok = self.fcds_de_ultima_busqueda is not None and not self.fcds_de_ultima_busqueda.empty
                rfd_ok = self.desc_finales_de_ultima_busqueda is not None and not self.desc_finales_de_ultima_busqueda.empty and self.origen_principal_resultados == OrigenResultados.VIA_DICCIONARIO_CON_RESULTADOS_DESC
                if fcd_ok or rfd_ok: messagebox.showwarning("Nada Seleccionado", "No seleccionó datos.", parent=dialog); decision['confirmed'] = False; return
            dialog.destroy()
        ttk.Button(frm_btns, text="Confirmar", command=on_ok).pack(side=tk.LEFT, padx=10)
        ttk.Button(frm_btns, text="Cancelar", command=dialog.destroy).pack(side=tk.LEFT, padx=10)

        self.update_idletasks()
        px, py, pw, ph = self.winfo_x(), self.winfo_y(), self.winfo_width(), self.winfo_height()
        dw, dh = dialog.winfo_reqwidth(), dialog.winfo_reqheight()
        dialog.geometry(f"+{px + (pw // 2) - (dw // 2)}+{py + (ph // 2) - (dh // 2)}")
        self.wait_window(dialog)
        return decision


    def _exportar_resultados(self): # Lógica de exportación se mantiene
        if not self.reglas_guardadas: messagebox.showwarning("Sin Reglas", "No hay reglas para exportar."); return

        ts_export = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
        f_sug = f"exportacion_reglas_{ts_export}.xlsx"
        ruta_save = filedialog.asksaveasfilename(title="Exportar Reglas...", initialfile=f_sug, defaultextension=".xlsx", filetypes=[("Excel", "*.xlsx")])
        if not ruta_save: self._actualizar_estado("Exportación cancelada."); return

        self._actualizar_estado("Exportando..."); num_r = len(self.reglas_guardadas)
        logging.info(f"Exportando {num_r} regla(s) a: {ruta_save}")
        try:
            with pd.ExcelWriter(ruta_save, engine='openpyxl') as writer:
                idx_export = []
                for i, regla in enumerate(self.reglas_guardadas):
                    tipo_short = regla.get('tipo_datos_guardados', 'DATOS').replace("RESULTADOS_DESCRIPCION_", "DESC_").replace("COINCIDENCIAS_DICCIONARIO", "FCD")[:10]
                    term_short = self._sanitizar_nombre_archivo(regla.get('termino_busqueda_original','S_T'),10)
                    id_hoja = f"R{i+1}_{term_short}_{tipo_short}"[:31]
                    idx_export.append({"ID_Regla_Hoja_Destino": id_hoja, "Termino_Busqueda_Original": regla.get('termino_busqueda_original', 'N/A'),
                                       "Operador_Principal_OR": regla.get('operador_principal_or', 'N/A'), "Tipo_Datos_Guardados": regla.get('tipo_datos_guardados', 'N/A'),
                                       "Fuente_Original_Resultados": regla.get('fuente_original_guardado', 'N/A'), "Timestamp_Guardado": regla.get('timestamp', 'N/A'),
                                       "Num_Filas_Snapshot": len(regla.get('datos_snapshot', []))})

                    df_def = pd.DataFrame([{'termino_original': regla.get('termino_busqueda_original'), 'operador_principal_or': regla.get('operador_principal_or'),
                                           'segmentos_parseados_json': json.dumps(regla.get('segmentos_parseados_para_and'), ensure_ascii=False, indent=2)}])
                    df_def.to_excel(writer, sheet_name=f"Def_{id_hoja}"[:31], index=False)

                    snap_list = regla.get('datos_snapshot')
                    if snap_list:
                        df_snap = pd.DataFrame(snap_list)
                        if not df_snap.empty: df_snap.to_excel(writer, sheet_name=id_hoja, index=False)
                if idx_export: pd.DataFrame(idx_export).to_excel(writer, sheet_name="Indice_Reglas_Exportadas", index=False)
            logging.info(f"Exportación de {num_r} regla(s) completada a {ruta_save}")
            messagebox.showinfo("Éxito", f"{num_r} regla(s) exportadas a:\n{ruta_save}")
            self._actualizar_estado(f"Reglas exportadas a {Path(ruta_save).name}.")
            if messagebox.askyesno("Limpiar Reglas", "Exportación exitosa.\n¿Limpiar reglas guardadas?"):
                self.reglas_guardadas.clear(); self._actualizar_estado("Reglas limpiadas.")
            self._actualizar_botones_estado_general()
        except Exception as e:
            logging.exception("Error exportando reglas."); messagebox.showerror("Error Exportar", f"No se pudo exportar:\n{e}")
            self._actualizar_estado("Error exportando reglas.")


    def _sanitizar_nombre_archivo(self, texto: str, max_len: int = 50) -> str: # Utilidad, se mantiene
        if not texto: return "resultados"
        import re # Asegurar importación de re si se usa aquí explícitamente
        sane = re.sub(r'[^\w\s-]', '', texto)
        sane = re.sub(r'[-\s]+', '_', sane).strip('_')
        return sane[:max_len]

    def _actualizar_estado_botones_operadores(self): # UI, se mantiene la lógica de habilitación
        if self.motor.datos_diccionario is None and self.motor.datos_descripcion is None:
            self._deshabilitar_botones_operadores(); return

        texto = self.texto_busqueda_var.get()
        for btn in self.op_buttons.values(): btn["state"] = "normal"

        cursor_pos = self.entrada_busqueda.index(tk.INSERT)
        antes_cursor = texto[:cursor_pos].strip()
        ultimo_char = antes_cursor[-1] if antes_cursor else ""

        puede_logico = bool(antes_cursor) and ultimo_char not in ['+', '|', '/', '#','<','>','=','-',' ']
        puede_nuevo_term = not antes_cursor or ultimo_char in ['+', '|', '/',' ']

        self.op_buttons["+"]["state"] = "normal" if puede_logico else "disabled"
        self.op_buttons["|"]["state"] = "normal" if puede_logico else "disabled"
        self.op_buttons["#"]["state"] = "normal" if puede_nuevo_term or antes_cursor.endswith(tuple(op+" " for op in ["+", "|", "/"])) else "disabled"

        puede_comp_rango = puede_nuevo_term or (antes_cursor and not re.search(r'[<>=\-]$', antes_cursor))
        for op_k in [">", "<", ">=", "<=", "-"]: # Asumiendo mapeo interno para '≥', '≤'
             self.op_buttons[op_k]["state"] = "normal" if puede_comp_rango else "disabled"

    def _insertar_operador_validado(self, operador: str): # UI, se mantiene
        if self.motor.datos_diccionario is None and self.motor.datos_descripcion is None: return

        txt_insert = operador
        if operador in ["+", "|", "/"]:
            prefijo = " " if not self.entrada_busqueda.get()[:self.entrada_busqueda.index(tk.INSERT)].endswith(" ") else ""
            txt_insert = f"{prefijo}{operador} "
        elif operador == "#": txt_insert = f"{operador}"

        self.entrada_busqueda.insert(tk.INSERT, txt_insert)
        self.entrada_busqueda.focus_set()

    def _deshabilitar_botones_operadores(self): # UI, se mantiene
        for btn in self.op_buttons.values(): btn["state"] = "disabled"

    def on_closing(self): # UI, se mantiene
        logger.info("Cerrando la aplicación...")
        self._guardar_configuracion()
        self.destroy()

---
## 6. `main.py` (Script Principal)

In [ ]:
# main.py
import tkinter as tk
from tkinter import messagebox
import pandas as pd # Para la verificación de versión y dependencia
import openpyxl # Para la verificación de versión y dependencia
import logging
from pathlib import Path

# Importar clases y configuraciones de los otros módulos
from interfaz_grafica import InterfazGrafica
from config import LOG_FILE_NAME # Usar la constante del archivo de configuración

# Configuración del logging (similar a la original, pero centralizada aquí)
logger = logging.getLogger() # Obtener el logger raíz
logger.setLevel(logging.INFO) # Nivel por defecto, puede ser cambiado por config

# Formatter
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(filename)s:%(lineno)d - %(message)s'
)

# File Handler
try:
    fh = logging.FileHandler(LOG_FILE_NAME, encoding='utf-8', mode='a')
    fh.setFormatter(formatter)
    logger.addHandler(fh)
except Exception as e:
    print(f"Error configurando FileHandler para logging: {e}")
    # No añadir handler si falla, para evitar problemas si no se puede escribir el log

# Stream Handler (consola)
sh = logging.StreamHandler()
sh.setFormatter(formatter)
logger.addHandler(sh)


def verificar_dependencias():
    """Verifica si las dependencias críticas están instaladas."""
    missing_deps = []
    try:
        logger.info(f"Pandas versión: {pd.__version__}")
    except ImportError:
        missing_deps.append("pandas")
        logger.critical("Dependencia faltante: pandas")
    except AttributeError: # pd puede ser importado pero __version__ no existir si es un mock o algo raro
        missing_deps.append("pandas (versión desconocida)")
        logger.critical("No se pudo obtener la versión de pandas.")


    try:
        # openpyxl no siempre tiene __version__ fácilmente accesible a través de import openpyxl; openpyxl.__version__
        # Es mejor solo comprobar la importación.
        import openpyxl as op_xl # Renombrar para evitar conflicto con la variable global en otros módulos si existiera
        logger.info("openpyxl importado correctamente.")
    except ImportError:
        missing_deps.append("openpyxl (para archivos .xlsx)")
        logger.warning("Dependencia opcional faltante: openpyxl. Funcionalidad para .xlsx limitada.")

    if "pandas" in missing_deps or "pandas (versión desconocida)" in missing_deps:
        error_msg_dep = (
            f"Faltan librerías críticas: {', '.join(d for d in missing_deps if 'pandas' in d)}.\n"
            f"Instale con: pip install pandas\n"
            f"Otras dependencias opcionales faltantes: {', '.join(d for d in missing_deps if 'openpyxl' in d)}"
        )
        logger.critical(error_msg_dep)
        try:
            root_temp = tk.Tk()
            root_temp.withdraw()
            messagebox.showerror("Dependencias Faltantes", error_msg_dep)
            root_temp.destroy()
        except tk.TclError:
            print(f"ERROR CRÍTICO (Tkinter no disponible): {error_msg_dep}")
        except Exception as e_tk_init:
            print(f"ERROR CRÍTICO (Error al mostrar msgbox): {e_tk_init}\n{error_msg_dep}")
        return False
    return True


if __name__ == "__main__":
    logger.info("=============================================")
    logger.info(f"=== Iniciando Aplicación Buscador ({Path(__file__).name}) ===")

    if not verificar_dependencias():
        exit(1)

    try:
        app = InterfazGrafica()
        app.mainloop()
    except Exception as main_error:
        logger.critical("¡Error fatal no capturado en la aplicación!", exc_info=True)
        try:
            root_err = tk.Tk()
            root_err.withdraw()
            messagebox.showerror("Error Fatal", f"Error crítico:\n{main_error}\nConsulte '{LOG_FILE_NAME}'.")
            root_err.destroy()
        except Exception as fallback_error:
            logger.error(f"No se pudo mostrar el mensaje de error fatal vía Tkinter: {fallback_error}")
            print(f"ERROR FATAL: {main_error}. Consulte {LOG_FILE_NAME}.")
    finally:
        logger.info(f"=== Finalizando Aplicación Buscador ({Path(__file__).name}) ===")

**Cómo Usar Esta Estructura:**

1.  Crea una carpeta para tu proyecto.
2.  Dentro de esa carpeta, crea los seis archivos Python (`config.py`, `enums.py`, `utilidades.py`, `motor_busqueda.py`, `interfaz_grafica.py`, `main.py`) y copia el código correspondiente en cada uno.
3.  Asegúrate de que todas las dependencias (`pandas`, `openpyxl`) estén instaladas en tu entorno Python.
4.  Ejecuta el programa desde el archivo `main.py`: `python main.py`

Esta organización te permitirá:

* Tener cada parte del sistema con una responsabilidad clara.
* Facilitar la modificación y prueba de componentes individuales.
* Mejorar la legibilidad general del proyecto.
* Gestionar las configuraciones de forma centralizada.

Espero que esta estructura te sea de gran utilidad. ¡Avísame si tienes alguna otra pregunta!